In [1]:
import tensorflow as tf

/home/haines/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/haines/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/haines/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
t0 = tf.constant(3, dtype=tf.int32)
t1 = tf.constant([3., 4.1, 5.2], dtype=tf.float32)
t2 = tf.constant([['Apple', 'Orange'],['Potato','Tomato']], dtype=tf.string)
t3 = tf.constant([[[5],[6],[7]],[[4],[3],[2]]])

print(t0)
print(t1)
print(t2)
print(t3)

Tensor("Const_2:0", shape=(), dtype=int32)
Tensor("Const_3:0", shape=(3,), dtype=float32)
Tensor("Const_4:0", shape=(2, 2), dtype=string)
Tensor("Const_5:0", shape=(2, 3, 1), dtype=int32)


In [7]:
sess = tf.Session()
print(sess.run(t3))

[[[5]
  [6]
  [7]]

 [[4]
  [3]
  [2]]]


In [9]:
node1 = tf.constant(3.1)
node2 = tf.constant(4.4)
adder = node1 + node2
print(sess.run(adder))

7.5


In [20]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_t = a + b
print(a)
print(adder_t)
print(sess.run(adder_t, {a:[1,2,3],b:[3,4,5]}))
add_and_triple = adder_t * 3
print(sess.run(add_and_triple, {a: [1,2,3,], b:[4,5,6]}))

Tensor("Placeholder_14:0", dtype=float32)
Tensor("add_9:0", dtype=float32)
[4. 6. 8.]
[15. 21. 27.]


In [23]:
w = tf.Variable([.1], dtype=tf.float32)
b = tf.Variable([-.2],dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = w*x+b

y = tf.placeholder(tf.float32)
loss = tf.reduce_sum(tf.square(linear_model - y))

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(linear_model,{x:[1,2,3,4,5,6]}))
print(sess.run(loss, {x:[1,2,3,4,5,6], y:[2,3,4,5,6,7]}))

[-0.1         0.          0.10000001  0.2         0.3         0.40000004]
127.71


In [29]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)


W: [1.0000045] b: [0.99998164] loss: 3.968239e-10
3.968239e-10
[1.9999862 2.9999907 3.9999952 5.        6.000005  7.0000086]


In [33]:
import numpy as np
def model_fn(features, labels, mode):
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W * features['x'] + b
    
    loss = tf.reduce_sum(tf.square( y - labels))
    
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.001)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    
    return tf.estimator.EstimatorSpec(mode=mode, predictions=y, loss=loss, train_op= train)
    
estimator = tf.estimator.Estimator(model_fn=model_fn)

x_train = np.array([1., 2., 3., 6., 8.])
y_train = np.array([4.8, 8.5, 10.4, 21.0, 25.3])

x_eavl = np.array([2., 5., 7., 9.])
y_eavl = np.array([7.6, 17.2, 23.6, 28.8])


train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=2, num_epochs=None, shuffle=True)

train_input_fn_2 = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=2, num_epochs=1000, shuffle=False)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eavl}, y_eavl, batch_size=2, num_epochs=1000, shuffle=False)

estimator.train(input_fn=train_input_fn, steps=1000)

train_metrics = estimator.evaluate(input_fn=train_input_fn_2)
print("train metrics: %r" % train_metrics)

eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("eval metrics: %s" % eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp91lttw1k', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f80781133c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp91lttw1k/model.ckpt.
INFO:ten